Import needed libraries

In [1]:
import os
import sys
from pathlib import Path

sys.path.append("..")

from utils.llm_client import LLMClient
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.login_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model


In [2]:
model = pick_model("groq", "reason")
llm_client = LLMClient("groq", model)

with open("../data/Incidents.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    headers = [h for h in lines[0].split("|")]

records = []
for line in lines[1:]:
    if not line.strip():
        continue
    values = [v.strip() for v in line.split("|")]
    record = dict(zip(headers, values))
    records.append(record)

for record in records:
    prompt_text, spec = render(
        "cot_reasoning.v2",
        role="SOS call analyzer",
        problem = record,
        logic = "Assign a Priority Score (1-10) based on Age Need and Urgency.Base Score: 5, +2 if Age > 60 or < 5, +3 if Need == Rescue (Life Threat), +1 if Need == Insulin/Medicine ",
        output = "Score: {score}/10"
    )

    messages = [{"role": "user", "content": prompt_text}]
    response = llm_client.chat(messages, temperature=1.0)

    response_text = response["text"]
    print(response_text)

## Reasoning Steps:

1. **Determine Base Score**: The base score is given as 5.
2. **Adjust for Age**: The ages provided are between 20-40. Since none of the ages are > 60 or < 5, no adjustment is made for age.
3. **Adjust for Need**: The main need is for water, not rescue or insulin/medicine. Therefore, no additional points are awarded for need.
4. **Assess Urgency**: The message indicates that the people are thirsty but safe on the roof with a stable water level. This suggests a moderate level of urgency but does not explicitly qualify for additional points based on the provided criteria.

## Answer:

Given the information:
- Base Score: 5
- Age Adjustment: 0 (since ages are between 20-40)
- Need Adjustment: 0 (since need is water, not rescue or life-threatening)
- Urgency Adjustment: 0 (no explicit criteria met for additional urgency points)

The final Priority Score is: **5/10**
## Reasoning Steps:

1. **Base Score**: Start with a base score of 5.
2. **Age Adjustment**: Check if th

In [ ]:
model = pick_model("groq", "reason")
llm_client = LLMClient("groq", model)

with open("../data/Incidents.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    headers = [h for h in lines[0].split("|")]

records = []
for line in lines[1:]:
    if not line.strip():
        continue
    values = [v.strip() for v in line.split("|")]
    record = dict(zip(headers, values))
    records.append(record)


prompt_text, spec = render(
    "tot_reasoning.v2",
    role="SOS call analyzer",
    branches= 3,
    problem = records,
    branching_logic = "Branch 1: Save the highest score first (Greedy),Branch 2: Save closest first (speed), Branch 3: Save furthest first (Logistics).",
)

messages = [{"role": "user", "content": prompt_text}]
response = llm_client.chat(messages, temperature=1.0)

response_text = response["text"]
print(response_text)

### Solution Paths for SOS Call Analysis

Given the SOS call: 
- **ID**: '1'
- **Time**: '08:00 AM'
- **Area**: 'Gampaha'
- **People**: '4'
- **Ages**: '20-40'
- **Main Need**: 'Water'
- **Message**: '"Thirsty but safe on roof. Water level stable."'

### Branch 1: Save the Highest Score First (Greedy)

- **Hypothesis**: Prioritizing saves based on a scoring system that includes factors like the number of people in need, their age (vulnerability), and the urgency of their need. Assuming a higher score means a more critical situation.
- **Steps**:
  1. Assign scores to each call based on the number of people (higher is more critical), age group (younger and older adults are more vulnerable), and urgency of need (water is critical).
  2. Calculate the score for the given SOS call: 
     - People: 4 (assuming a linear scale, 4 people = 4 points)
     - Ages: 20-40 (less vulnerable, 2 points)
     - Main Need: Water (critical, 3 points)
     - Total Score: 4 (people) + 2 (ages) + 3 (need) =